In [104]:
using JuMP
using GLPK
using SDDP
using HiGHS
using MAT
using CSV
using DataFrames
using Distributions


model = Model(HiGHS.Optimizer)

# Load the .mat file
mat_data_0 = matread("C:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/Senario generation/zeta_0.mat")
ζ0 = mat_data_0["zeta_0"]
# Load the .mat file
mat_data_1 = matread("c:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/Senario generation/zeta_1.mat")
ζ1 = mat_data_1["zeta_1"]
# Load the .mat file
CR_mat = matread("c:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/Senario generation/CR.mat")
Ch_mat = matread("c:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/Senario generation/Ch.mat")

# Load the Excel file
df = CSV.read("C:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/optimisation/IEEE 33.csv",DataFrame)

println("i stopped here")
# Convert DataFrame columns to arrays for easier use
branch_no = df[!, "Branch No"]
sending_end = df[!, "Sending End"]
receiving_end = df[!, "Receiving End"]
R = df[!, "R (ohms)"]
X = df[!, "X (ohms)"]
PL = df[!, "PL (kW)"]
PL = [0;PL]
QL = df[!, "QL (kVAR)"]
QL = [0;QL]
println("i stopped here")
# Define sets
ΩN = 1:33  # Example set of buses
ΩB = 1:33  # Example set of lines
#ΩDG = 1:5  # Example set of buses with DGs
#ΩSW = 1:10  # Example set of lines with switches
T = 1:12  # Example set of time periods
S = 1:9  # Example set of scenarios


# Define parameters (replace these with actual values)
#cc = Dict((i, j) => 10.0 for i in ΩB, j in ΩB) #annual capital cost for adding an automatic switch
ch = Ch_mat["Ch"] #hardening lines
#cg = Dict(i => 100.0 for i in ΩDG) #deploying a back-up DG
cL = Dict(i => 14.0 for i in ΩN) #penalty cost for load-shedding
cR = CR_mat["CR"] #base repair cost
ωH = 0.01 #average occurance per year
pr = fill(1.0/length(S), length(S)) #every senario havinf same probabilities
#pr = Dict(s => 0.1 for s in S)  # Example scenario probabilities
#PL = Dict((i, t, s) => rand(100:200) for i in ΩN, t in T, s in S)  # Example loads
#QL = Dict((i, t, s) => rand(50:100) for i in ΩN, t in T, s in S)  # Example reactive loads


# Define first-stage variables
#@variable(model, xg[i in ΩDG], Bin) #new back-up DG
@variable(model, xh[i in ΩB], Bin) #hardening
#@variable(model, xc[i in ΩB, j in ΩB], Bin) #line switch
#@variable(model, xc1[i in ΩB, j in ΩB], Bin) #new line switch


# Define second-stage variables
#@variable(model, γ[i in ΩN, t in T, p in P, s in S] >= 0)  # Load shedding percentage
@variable(model, u[i in ΩB, t in T, s in S], Bin)  # Line damage status
println("I stop at second-stage cost variables")
# Define second-stage cost components
@expression(model, repair_cost[s in S], sum(cR[i] * u[i, 12, s] for i in ΩB, s in S))
#@expression(model, load_shedding_cost, sum(cL[i] * γ[i, t, s] * PL[i] for i in ΩN, t in T, s in S)) #key not found
println("I stop at second-stage cost components")

# Objective function
@objective(model, Min, sum(ch[i] * xh[i] for i in ΩB) +
                        #sum(cg[i] * xg[i] for i in ΩDG) +
                        #sum(cc[i, j] * xc1[i, j] for i in ΩB, j in ΩB) +
                        ωH * sum(pr[s] * (repair_cost[s]) for s in S))

# Constraints
println("Hello! I passed Objective function")
#@constraint(model, sum(xg[i] for i in ΩDG) <= length(ΩDG))  # Example constraint for DGs
#@constraint(model, [i in ΩB, j in ΩB], xc[i, j] == xc1[i, j])  # Example constraint for switches
@constraint(model, [i in ΩB, t in T, s in S],
    u[i, t, s] == (1 - xh[i]) * ζ0[i, t, s] + xh[i] * ζ1[i, t, s])

# Add other constraints for power flow, voltage limits, and so on


println("where is my constraint")
# Solve the model
optimize!(model)

i stopped here
i stopped here
I stop at second-stage cost variables
I stop at second-stage cost components
Hello! I passed Objective function
where is my constraint
Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+01, 2e+04]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
0 rows, 0 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Solving report
  Status            Optimal
  Primal bound      4843.8
  Dual bound        4843.8
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    4843.8 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.02 (total)
                    0.01 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (s

In [105]:
# Print the results
println("Objective value: ", objective_value(model))
for i in ΩB
    if xh[i] != 0.0
        println("xh[$i] = ", value(xh[i]))  
    end
end


Objective value: 4843.8
xh[1] = 0.0
xh[2] = 0.0
xh[3] = 0.0
xh[4] = 0.0
xh[5] = 0.0
xh[6] = 0.0
xh[7] = 0.0
xh[8] = 0.0
xh[9] = 0.0
xh[10] = 0.0
xh[11] = 0.0
xh[12] = 0.0
xh[13] = 0.0
xh[14] = 0.0
xh[15] = 0.0
xh[16] = 0.0
xh[17] = 0.0
xh[18] = 0.0
xh[19] = 0.0
xh[20] = 0.0
xh[21] = 0.0
xh[22] = 0.0
xh[23] = 0.0
xh[24] = 0.0
xh[25] = 0.0
xh[26] = 0.0
xh[27] = 0.0
xh[28] = 0.0
xh[29] = 0.0
xh[30] = 0.0
xh[31] = 0.0
xh[32] = 0.0
xh[33] = 0.0


In [25]:
damage = 0
for i in ΩB, t in 12, s in S
    if value(u[i, t, s] )> 0.1
       damage = damage +1 
    end
end
println(value(damage),"damages are made")

3889damages are made


In [106]:
using JuMP
using GLPK
using SDDP
using HiGHS
using MAT
using CSV
using DataFrames
using Distributions
using Gurobi

harden_model = Model(Gurobi.Optimizer)

# Load the .mat files
mat_data_0 = matread("C:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/Senario generation/zeta_0.mat")
ζ0 = mat_data_0["zeta_0"]

mat_data_1 = matread("C:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/Senario generation/zeta_1.mat")
ζ1 = mat_data_1["zeta_1"]

# Load the .mat file
CR_mat = matread("c:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/Senario generation/CR.mat")
Ch_mat = matread("c:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/Senario generation/Ch.mat")
load_mat = matread("c:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/Senario generation/load.mat")

# Load the Excel file
df = CSV.read("C:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/optimisation/IEEE 33.csv", DataFrame)

# Convert DataFrame columns to arrays for easier use
branch_no = df[!, "Branch No"]
sending_end = df[!, "Sending End"]
receiving_end = df[!, "Receiving End"]
R = df[!, "R (ohms)"]
X = df[!, "X (ohms)"]
PL = df[!, "PL (kW)"]
PL = [0; PL]
QL = df[!, "QL (kVAR)"]
QL = [0; QL]


# Define sets
ΩN = 1:33  # Example set of buses
ΩB = 1:32  # Example set of lines
T = 1:12  # Example set of time periods
S = 1:90  # Example set of scenarios

# Define parameters (replace these with actual values)
ch = Ch_mat["Ch"] #hardening lines
cL = 14.0 #penalty cost for load-shedding
cR = CR_mat["CR"] #base repair cost
ωH = 0.1 #average occurrence per year
pr = fill(1.0/length(S), length(S)) #every scenario having the same probabilities
P = load_mat["P_total"]

# Define first-stage variables
@variable(harden_model, xh[i in ΩB], Bin) #hardening

# Define second-stage variables
@variable(harden_model, u[i in ΩB, t in T, s in S], Bin)  # Line damage status

# Define second-stage cost components
@expression(harden_model, repair_cost[s in S], sum(cR[i] * u[i, 12, s] for i in ΩB))

# Objective function
@objective(harden_model, Min, sum(ch[i] * xh[i] for i in ΩB) + ωH * sum(pr[s] * (repair_cost[s]) for s in S))

# Constraints
@constraint(harden_model, [i in ΩB, t in T, s in S],
    u[i, t, s] == (1 - xh[i]) * ζ0[i, t, s] + xh[i] * ζ1[i, t, s])
# Solve the model



optimize!(harden_model)



Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2547371
Academic license 2547371 - for non-commercial use only - registered to jo___@imperial.ac.uk


KeyError: KeyError: key "P_total" not found

32415.55555555556 when xh[1] =1

In [3]:
println("Objective value: ", objective_value(harden_model))

for i in ΩB
    if value(xh[i]) >= 0.01
        println("xh[$i] = ", value(xh[i]))  
    end
end

println("expectation:", value(sum((pr[s] * repair_cost[s]) for s in S)))


println(γ[2,2])
println(gamma)


Objective value: 72635.55555555555
xh[17] = 1.0
expectation:695555.5555555555


UndefVarError: UndefVarError: `γ` not defined

In [49]:
using JuMP
using HiGHS
using Graphs
using CSV
using DataFrames
using MAT
using Gurobi

# Load the Excel file
df = CSV.read("C:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/optimisation/IEEE 33.csv", DataFrame)

# Load the .mat files
CR_mat = matread("c:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/Senario generation/CR.mat")
Ch_mat = matread("c:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/Senario generation/Ch.mat")
load_mat = matread("c:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/Senario generation/load.mat")
mat_data_0 = matread("C:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/Senario generation/zeta_0.mat")
ζ0 = mat_data_0["zeta_0"]
mat_data_1 = matread("c:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/Senario generation/zeta_1.mat")
ζ1 = mat_data_1["zeta_1"]

# Convert DataFrame columns to arrays for easier use
sending_end = df[!, "Sending End"]
receiving_end = df[!, "Receiving End"]
#PL = df[!, "PL (kW)"]
#PL = [0; PL]  # Prepend a 0 for consistency with indexing #PL[i] = load at node i
PL = load_mat["PL"]

P_max = 6330
P_g_max = 200
#QL = df[!, "QL (kVAR)"]
#QL = [0; QL]  # Prepend a 0 for consistency with indexing
QL = load_mat["QL"]
time = load_mat["time"]
time = convert.(Int, time)



R = df[!, "R(pu)"]
X = df[!, "X(pu)"]
Q_max = 2195
Q_g_max = 50
V0 = 1 #reference voltage kV

# Define sets
ΩN = 1:33  # Set of buses
ΩSG = [1]  # Set of substation buses 
ΩDG = 2:33  # Set of buses with DGs
ΩL = 2:33   # Set of buses with loads
ΩB = 1:32  # Set of lines
T = 1:24  # Set of time periods
S = 1:90 # Set of scenarios


# Define parameters (replace these with actual values)
ch = Ch_mat["Ch"]  # Hardening lines
cL = 16.0  # Penalty cost for load-shedding # $10.99 in 2009
cR = CR_mat["CR"]  # Base repair cost
cg = 38080 #/kW annual
ωH = 0.2  # Average occurrence per year
pr = fill(1.0/length(S), length(S))  # Every scenario having the same probabilities
M = 99999999

# Define the optimization model
test = Model(Gurobi.Optimizer)

# Define first-stage variables (xh: hardening decision)
@variable(test, xh[i in ΩB], Bin)
@variable(test, xg[i in ΩDG], Bin)

# Define second-stage variables (u: line damage status)
@variable(test, u[i in ΩB, t in T, s in S], Bin)
@variable(test, P[i in ΩB, t in T, s in S])  # Active power flow
@variable(test, Q[i in ΩB, t in T, s in S])  # Reactive power flow
@variable(test, γ[i in ΩN, t in T, s in S] >= 0)  # Load shedding percentage
@variable(test, P_g[i in ΩN, t in T, s in S] >= 0)  # Active power generation
@variable(test, Q_g[i in ΩN, t in T, s in S] >= 0)  # Reactive power generation
@variable(test, V[i in ΩN, t in T, s in S])

# Define second-stage cost components
@expression(test, repair_cost[s in S], sum(cR[i] * u[i, t, s] for i in ΩB, t in T))
@expression(test, load_shedding_cost[s in S], sum(cL * γ[i, t, s] * PL[i, time[t, s]] for i in ΩB, t in T))


# Objective function
@objective(test, Min, sum(ch[i] * xh[i] for i in ΩB) + sum(cg * xg[j] for j in ΩDG) + ωH * sum(pr[s] * (repair_cost[s] + load_shedding_cost[s]) for s in S))#

# Constraints to model line damage status
@constraint(test, [i in ΩB, t in T, s in S],
    u[i, t, s] == (1 - xh[i]) * ζ0[i, t, s] + xh[i] * ζ1[i, t, s])

# Constraint (13): Active power balance
@constraint(test, [i in ΩN, t in T, s in S],
    sum(P[j, t, s] for j in ΩB if sending_end[j] == i) - # Search through IEEE 33 
    sum(P[j, t, s] for j in ΩB if receiving_end[j] == i) ==
    P_g[i, t, s] - (1 - γ[i, t, s]) * PL[i, time[t,s]]
)
# Constraint (14): Reactive power balance
@constraint(test, [i in ΩN, t in T, s in S],
    sum(Q[j, t, s] for j in ΩB if sending_end[j] == i) -
    sum(Q[j, t, s] for j in ΩB if receiving_end[j] == i) ==
    Q_g[i, t, s] - (1 - γ[i, t, s]) * QL[i, time[t,s]]
)
println("I passed this constraint")

# Constraint (18): 
@constraint(test, [i in ΩB, t in T, s in S],
    P[i, t, s] <= (1 - u[i, t, s]) * P_max
)
@constraint(test, [i in ΩB, t in T, s in S],
    P[i, t, s] >= (u[i, t, s] - 1) * P_max
)

# Constraint (19): 
@constraint(test, [i in ΩB, t in T, s in S],
    Q[i, t, s] <= (1 - u[i, t, s]) * Q_max
)
@constraint(test, [i in ΩB, t in T, s in S],
    Q[i, t, s] >= (u[i, t, s] - 1) * Q_max
)

# Constraint (21): voltage drop limit
@constraint(test, [i in ΩB, t in T, s in S],
    V[sending_end[i], t, s] - (R[i] * P[i, t, s]/1000 + X[i] * Q[i, t, s]/1000) /V0 - u[i, t, s] * M <= V[receiving_end[i], t, s] #pu
)
@constraint(test, [i in ΩB, t in T, s in S],
    V[sending_end[i], t, s] - (R[i] * P[i, t, s]/1000 + X[i] * Q[i, t, s]/1000) /V0 + u[i, t, s] * M >= V[receiving_end[i], t, s]
)

# Constraint (27) generation limit
@constraint(test, [i in ΩDG, t in T, s in S], P_g[i, t, s] <= xg[i] * P_g_max)
# Constraint (28)
@constraint(test, [i in ΩDG, t in T, s in S], Q_g[i, t, s] <= xg[i] * Q_g_max)

# Implement Constraint (29): Non-negativity of active power generation
@constraint(test, [i in ΩSG, t in T, s in S], P_g[i, t, s] >= 0)



# Implement Constraint (30): Non-negativity of reactive power generation
@constraint(test, [i in ΩSG, t in T, s in S], Q_g[i, t, s] >= 0)


# Ensure voltage is within [0.9, 1.1]pu
@constraint(test, [i in ΩN, t in T, s in S], 1.1 * V0 >= V[i, t, s] >= 0.9 * V0)


# Ensure load shedding percentage is within [0, 1]
@constraint(test, [i in ΩN, t in T, s in S], 1 >= γ[i, t, s])

println("all constraints passed")

# Solve the model
optimize!(test)


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2547371
Academic license 2547371 - for non-commercial use only - registered to jo___@imperial.ac.uk
I passed this constraint
all constraints passed
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900HX, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Academic license 2547371 - for non-commercial use only - registered to jo___@imperial.ac.uk
Optimize a model with 911520 rows, 563824 columns and 2384116 nonzeros
Model fingerprint: 0x7a42702c
Variable types: 494640 continuous, 69184 integer (69184 binary)
Coefficient statistics:
  Matrix range     [3e-07, 1e+08]
  Objective range  [9e-01, 4e+04]
  Bounds range     [9e-01, 1e+00]
  RHS range        [1e+00, 6e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

In [47]:
for i in ΩB
    println("γ [$i] = ", value(γ[i,1,3]))
    println("u [$i] = ", value(u[i,1,3]))
    println("voltage drop at line $i = ", value((R[i] * P[i, 1, 3]/1000 + X[i] * Q[i, 1, 3]/1000) /V0))
    println("P [$i] = ",value(P[i,1,3]))
     
end

γ [1] = 0.0
u [1] = 0.0
voltage drop at line 1 = 0.00245276725
P [1] = 3399.15
γ [2] = 0.0
u [2] = 0.0
voltage drop at line 2 = 0.011448667999999999
P [2] = 2971.35
γ [3] = 0.0
u [3] = 0.0
voltage drop at line 3 = 0.00576897
P [3] = 2022.75
γ [4] = 0.0
u [4] = 0.0
voltage drop at line 4 = 0.005663552
P [4] = 1911.15
γ [5] = 0.0
u [5] = 0.0
voltage drop at line 5 = 0.0134189685
P [5] = 1855.35
γ [6] = 0.0
u [6] = 0.0
voltage drop at line 6 = 0.0030398775000000005
P [6] = 999.75
γ [7] = 0.0
u [7] = 0.0
voltage drop at line 7 = 0.011710169999999999
P [7] = 813.75
γ [8] = 0.0
u [8] = 0.0
voltage drop at line 8 = 0.0053970225
P [8] = 627.75
γ [9] = 0.0
u [9] = 0.0
voltage drop at line 9 = 0.0049847655
P [9] = 571.95
γ [10] = 0.0
u [10] = 0.0
voltage drop at line 10 = 0.0007390035
P [10] = 516.15
γ [11] = 0.0
u [11] = 0.0
voltage drop at line 11 = 0.001285878
P [11] = 474.3
γ [12] = 0.0
u [12] = 0.0
voltage drop at line 12 = 0.005237607500000001
P [12] = 418.5
γ [13] = 0.0
u [13] = 0.0
volta

In [35]:
for i in ΩN
    println("V[$i] = ",value(V[i,1,3]))
    println("P_g[$i] = ",value(P_g[i,1,3]))
    println("power out = ", value(sum(P[j, 1, 3] for j in ΩB if sending_end[j] == i)))
    println("power in = ", value(sum(P[j, 1, 3] for j in ΩB if receiving_end[j] == i)))
    println("served load = ",value((1 - γ[i, 1, 3]) * PL[i, time[1,3]]))
end


V[1] = 1.1
P_g[1] = 166.34718169005322
power out = 166.34718169005322


MethodError: MethodError: reducing over an empty collection is not allowed; consider supplying `init` to the reducer

In [50]:
Objective = objective_value(test)
println("Objective value: ", Objective) 



for i in ΩB
    if value(xh[i]) >= 0.01
        println("xh[$i] = ", value(xh[i]))  
    end
end

for i in ΩDG
    if value(xg[i]) >= 0.01
        println("xg[$i] = ", value(xg[i]))  
    end
end


ExpectedRepair = ωH * sum((pr[s] * repair_cost[s]) for s in S)
ExpectedLoadShedding = ωH * sum((pr[s] * load_shedding_cost[s]) for s in S)
Hardening_cost = sum(ch[i] * xh[i] for i in ΩB)
Generation_cost= sum(cg * xg[i] for i in ΩDG)

println("Repair expectation:", value(ExpectedRepair))
println("LoadShedding expectation:", value(ExpectedLoadShedding))
println("Hardening cost:", value(Hardening_cost))
println("Generation cost:",value(Generation_cost))

Objective value: 437977.85866666137
xh[1] = 1.0
xh[2] = 1.0
xh[18] = 1.0
Repair expectation:336593.4000000073
LoadShedding expectation:89984.45866666605
Hardening cost:11400.0
Generation cost:0.0


Objective value: 636083.4816140113
xh[1] = 1.0
xh[2] = 1.0
xh[3] = 1.0
xh[4] = 1.0
xh[5] = 1.0
xh[6] = 1.0
xh[7] = 1.0
xh[8] = 1.0
xh[9] = 1.0
xh[10] = 1.0
xh[11] = 1.0
xh[12] = 1.0
xh[13] = 1.0
xh[14] = 1.0
xh[15] = 1.0
xh[16] = 1.0
xh[17] = 1.0
xh[18] = 1.0
xh[19] = 1.0
xh[20] = 1.0
xh[21] = 1.0
xh[22] = 1.0
xh[23] = 1.0
xh[24] = 1.0
xh[25] = 1.0
xh[26] = 1.0
xh[27] = 1.0
xh[28] = 1.0
xh[29] = 1.0
xh[30] = 1.0
xh[31] = 1.0
xh[32] = 1.0
Repair expectation:115740.30000000034
LoadShedding expectation:211023.18161400364
Hardening cost:309320.0
Generation cost:0.0


Objective value: 559669.3274751984
Repair expectation:345879.30000000825
LoadShedding expectation:213790.02747519783
Hardening cost:0.0
Generation cost:0.0

Objective value: 557723.3806187133
xh[1] = 1.0
xh[18] = 1.0
Repair expectation:342625.40000000794
LoadShedding expectation:211297.9806187129
Hardening cost:3800.0
Generation cost:0.0

Objective value: 3.2348303992701895e6
xh[1] = 1.0
xh[2] = 1.0
xh[3] = 1.0
xh[4] = 1.0
xh[5] = 1.0
xh[6] = 1.0
xh[7] = 1.0
xh[8] = 1.0
xh[9] = 1.0
xh[10] = 1.0
xh[11] = 1.0
xh[12] = 1.0
xh[13] = 1.0
xh[14] = 1.0
xh[15] = 1.0
xh[16] = 1.0
xh[17] = 1.0
xh[18] = 1.0
xh[19] = 1.0
xh[20] = 1.0
xh[21] = 1.0
xh[22] = 1.0
xh[23] = 1.0
xh[24] = 1.0
xh[25] = 1.0
xh[26] = 1.0
xh[27] = 1.0
xh[28] = 1.0
xh[29] = 1.0
xh[30] = 1.0
xh[31] = 1.0
xg[3] = 1.0
xg[4] = 1.0
xg[5] = 1.0
xg[6] = 1.0
xg[7] = 1.0
xg[8] = 1.0
xg[9] = 1.0
xg[10] = 1.0
xg[11] = 1.0
xg[12] = 1.0
xg[13] = 1.0
xg[14] = 1.0
xg[15] = 1.0
xg[16] = 1.0
xg[17] = 1.0
xg[18] = 1.0
xg[19] = 1.0
xg[20] = 1.0
xg[21] = 1.0
xg[22] = 1.0
xg[23] = 1.0
xg[24] = 1.0
xg[25] = 1.0
xg[26] = 1.0
xg[27] = 1.0
xg[28] = 1.0
xg[29] = 1.0
xg[30] = 1.0
xg[31] = 1.0
xg[32] = 1.0
Repair expectation:119726.10000000137
LoadShedding expectation:524354.2992701962
Hardening cost:304000.0
Generation cost:2.28675e6



Objective value: 1.9586981772768788e7
Repair expectation:345879.30000000825
LoadShedding expectation:1.924110247276882e7
Hardening cost:0.0
Generation cost:0.0
No hardening

Objective value: 1.936280708355946e7
xh[1] = 1.0
xh[2] = 1.0
xh[18] = 1.0
Repair expectation:336593.4000000073
LoadShedding expectation:1.899201368355935e7
Hardening cost:34200.0
Generation cost:0.0

Selective hardening results

In [30]:
using Plots
# Example plot assuming scalar values (bar chart for total costs)
bar(
    ["Overall\nCost", "Expected\nRepair", "Expected\nLoad-shedding\nPenalty", "Hardening\nCost", "Generation\nCost"],
    [Objective/1000, value(ExpectedRepair)/1000, value(ExpectedLoadShedding)/1000, value(Hardening_cost)/1000, value(Generation_cost)/1000],
    label = "",
    title = "Cost Breakdown If All Branches Hardened",
    xlabel = "Cost Type",
    ylabel = "Cost (1000\$)",
    legend = false,
    dpi = 300,
    color = RGB(0/255,0/255, 205/255),  # Custom colors for the bars
    linecolor = false,
    xticks = ([0.5 1.5 2.5 3.5 4.5], ["Overall\nCost", "Expected\nRepair", "Expected\nLoad-shedding Cost", "Hardening\nCost", "Generation\nCost"]), # Rotate labels by 45 degrees
)
savefig("cost_sel_hardening.png")  # Save with high resolution (300 dpi)

"c:\\Users\\HP\\OneDrive - Imperial College London\\Planning for Extreme Weather event\\optimisation\\cost_sel_hardening.png"

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2547371
Academic license 2547371 - for non-commercial use only - registered to jo___@imperial.ac.uk
I passed this constraint
all constraints passed
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900HX, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Academic license 2547371 - for non-commercial use only - registered to jo___@imperial.ac.uk
Optimize a model with 455760 rows, 281945 columns and 1194650 nonzeros
Model fingerprint: 0x727800c2
Variable types: 247320 continuous, 34625 integer (34625 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+08]
  Objective range  [8e-01, 7e+03]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 6e+03]
Presolve removed 359215 rows and 186680 columns
Presolve time: 2.05s
Presolved: 96545 rows, 95265 columns, 336175 nonzeros
Variable types: 95201 continuous, 64 integer (64 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
...
Optimal solution found (tolerance 1.00e-04)
Best objective 1.325587956352e+05, best bound 1.325587956352e+05, gap 0.0000%

User-callback calls 659813, time in user-callback 0.84 sec

Output at 90 senarios
 Adjust cell output settings...
Objective value: 132558.79563520296
xh[1] = 1.0
xh[20] = 1.0
xg[4] = 1.0
xg[7] = 1.0
xg[8] = 1.0
xg[10] = 1.0
xg[14] = 1.0
xg[16] = 1.0
xg[18] = 1.0
xg[20] = 1.0
xg[22] = 1.0
xg[23] = 1.0
xg[24] = 1.0
xg[25] = 1.0
xg[29] = 1.0
xg[31] = 1.0
xg[32] = 1.0
expectation:751644.444444447
expectation:147178.85251558028
Hardening cost:2200.0
Generation cost:22500.0

In [58]:
println(value(γ[1,1,2]))
println(value(γ[2,1,2]))
println(value(P_g[1,1,2]))
println(value(P[1,1,2]))
println("P[2,1,2]:",value(P[2,1,2]))
println("P[2,1,2]:",value(P[3,1,2]))
println(value(P[32,1,2]))
println(value(P_g[2,1,2]))
println(value(sum(cL * γ[i, t, 2] * P[i, t, 2] for i in ΩB, t in T)))


0.0
0.0
100.0
100.0
P[2,1,2]:0.0
P[2,1,2]:0.0
0.0
0.0
0.0


In [ ]:
using Plots

Repair_expectation= 751644.444444447 * 0.2
LoadShedding_expectation = 147178.85251558028 * 0.2
Hardening_cost = 3800.0
Generation_cost = 116250
Objective_value = Repair_expectation + LoadShedding_expectation + Hardening_cost +Generation_cost

# Example plot assuming scalar values (bar chart for total costs)
bar(
    ["Overall\nCost", "Expected\nRepair", "Expected\nLoad-shedding\nPenalty", "Hardening\nCost", "Generation\nCost"],
    [Objective_value/1000, value(Repair_expectation)/1000, value(LoadShedding_expectation)/1000, value(Hardening_cost)/1000, value(Generation_cost)/1000],
    label = "",
    title = "Cost Breakdown If All Branches Hardened",
    xlabel = "Cost Type",
    ylabel = "Cost (1000\$)",
    legend = false,
    dpi = 300,
    color = RGB(0/255,0/255, 205/255),  # Custom colors for the bars
    linecolor = false,
    xticks = ([0.5 1.5 2.5 3.5 4.5], ["Overall\nCost", "Expected\nRepair", "Expected\nLoad-shedding Cost", "Hardening\nCost", "Generation\nCost"]), # Rotate labels by 45 degrees
)
savefig("cost_gene.png")  # Save with high resolution (300 dpi)

In [ ]:
using GraphPlot
using Graphs
# Create the base graph representing the IEEE 33-bus system
graph = SimpleGraph(length(unique([sending_end; receiving_end])))

# Add edges representing the branches between buses
for (s, r) in zip(sending_end, receiving_end)
    add_edge!(graph, s, r)
end
println("Graph has $(nv(graph)) nodes and $(ne(graph)) edges.")


gplot(graph)